<a href="https://colab.research.google.com/github/limseo12/deep-learning-from-scratch_Study/blob/main/%EB%B0%91%EB%B0%94%EB%8B%A5%EB%B6%80%ED%84%B0%EC%8B%9C%EC%9E%91%ED%95%98%EB%8A%94%EB%94%A5%EB%9F%AC%EB%8B%9D3_%EC%A0%9C1%EA%B3%A0%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Variable = 변수 , 클래스 구현
class Variable:
  def __init__(self, data):
    self.data = data

import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

#새로운 데이터입력
x.data = np.array(2.0)
print(x.data)

In [ ]:
#ndim 인스턴스 변수 사용해보기 = 차원 수
import numpy as np
x = np.array(1)
x.ndim
#결과 = 0
x = np.array([1, 2, 3])
x.ndim
#결과 = 1
x = np.array([[1, 2, 3],
              [4, 5, 6]])
x.ndim
#결과 = 2

In [ ]:
#2.2 Function 클래스 구현
class Function:
  def __call__(self, input):  #파이썬의 특수 메서드.
    x = input.data
    y = x ** 2
    output = Variable(y)
    return output

#2.3 FUnction 클래스 이용 2.2와 연계
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

In [ ]:
#calss Function 수정
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    return output
  def forward(self, x):
    raise NotImplementedError()

In [ ]:
#Function 클래스를 상속하여 입력값을 제곱하는 클래스를 구현
class Square(Function):
  def forward(self, x):
    return x ** 2

x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

## step 3 함수 연결

In [ ]:
#3.1 Exp 함수 구현
class Exp(Function):
  def forward(self, x):
    return np.exp(x)

#3.2 함수 연결 y = (e^x^2)^2 이라는 계산을 예로 생각
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

# Step 4 수치 미분

#중앙차분, 전진차분 .
#중앙차분이 상대적으로 오차가 적다
#앱실론의 약어 eps. 아주 작은 양의 부동소수점값을 담는 변수의 이름으로 흔희 사용된다.

In [ ]:
#4,1 중앙차분을 이용하여 수치미분을 계산해보자
def numerical_diff(f, x, eps=1e-4):
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (2 * eps)

# 아까 구현한 Square 클래스 대상으로 미분해보자
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

In [ ]:
#4.3 합성 함수의 미분
def f(x):
  A = Square()
  B = Exp()
  C = Square()
  return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)


4.4 수치미분의 문제점\
'자릿수 누락' 때문에 문제가있다.\
수치미분을 이용하면 자릿수 누락이 발생하여 오차가 포함되기 쉽다.

역전파\
수치 미분은 구현하기 쉽고 거의 정확한 값을 얻을 수 있다.\
이에 비해 역전파는 복잡한 알고리즘이여서 구현하면서 버그가 섞여 들어가기 쉽다.\
역전파를 정확하게 구현했는지 확인하기 위해 수치 미분의 결과를 이용하곤 한다.\
이를 기울기 확인 (gradient checking) 이라고한다\
이것은 단순히 수치 미분 결과와 역전파의 결과를 비교하는 것이다.

# Step 5 역전파 (Back propagation) 이론 

In [ ]:
#5.1 연쇄법칙


# Step 6 수동 역전파

In [ ]:
#6.1
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None  #grad 는 gradient 를 줄인 말

In [ ]:
#6.2 Function 클래스 추가 구현
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(x)
    self.input = input  # 입력 변수를 기억(보관)한다.
    return output
  
  def forward(self, x):
    raise NotImplementedError()
  
  def backward(self, gy):
    raise NotImplementedError()

In [ ]:
#6.3 Square와 Exp 클래스 추가 구현
class Square(Function):
  def foraward(self, x):
    y = x **2
    return y
  
  def backward(self, gy):
    x = self.input.data
    gx = 2 * x * gy
    return gx

In [ ]:
class Exp(Function):
  def forward(self, x):
    y = np.exp(x)
    return y
  
  def backward(self, gy):
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

In [ ]:
# 6.4 역전파 구현
#순전파하는 코드
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

#역전파로 y를 미분
y.grad = np.array(1.0)
b.grad = C.backward(b.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

# Step 7 역전파 자동화

In [ ]:
#7.1 역전파 자동화의 시작
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

In [ ]:
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    output.set_creator(self)
    self.input = input
    self.output = output
    return output

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

#계산 그래프의 노드들을 거꾸로 거슬러 올라간다.
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [ ]:
#7.2 역전파 도전!
y.grad = np.array(1.0)

C = y.creator # 1. 함수를 가져온다.
b = C.input # 2. 함수의 입력을 가져온다.
b.grad = C.backward(y.grad) # 3. 함수의 backward 메서드를 호출한다.

In [ ]:
B = b.creator #1. 함수를 가져온다
a = B.input #2. 함수의 입력을 가져온다.
a.grad = B.backward(b.grad) # 3. 함수의 backward 메서드를 호출한다.

In [ ]:
A = a.creator # 1. 함수를 가져온다.
x = A.input # 2. 함수의 입력을 가져온다.
x.grad = A.backward(a.grad) # 3. 함수의 backward 메서드를 호출한다.
print(x.grad)

In [ ]:
#7.3 backward 메서드 추가
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func
  
  def backward(self):
    f = self.creator  # 1. 함수를 가져온다.
    if f is not None:
      x = f.input # 2. 함수의 입력을 가져온다.
      x.grad = f.backward(self.grad)  # 3. 함수의 backward 메서드를 호출한다
      x.backward()  #하나 앞 변수의 backward 메서드를 호출한다(재귀)

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

#역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

# Setp 8 재귀에서 반복문으로

In [ ]:
# 8.1 현재의 Variable 클래스
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    f = self.creator
    if f is not None:
      x = f.input
      x.grad = f.vackward(self.grad)
      x.vackward()

In [ ]:
#8.2 반복문을 이용한 구현
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    funcs = [self.creator]
    while funcs:
      f = funcs.pop() # 함수를 가져온다
      x, y = f.input, f.output # 함수의 입력과 출력을 가져온다.
      x. grad = f.backward(y.grad)  # backward 메서드를 호출한다.

      if x.creator is not None:
        funcs.append(x.creator) # 하나 앞의 함수를 리스트에 추가한다.

In [ ]:
# 8.3 동작 확인
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

#역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

# Step 9 함수를 더 편리하게

In [ ]:
#9.1 파이썬 함수로 이용하기
x = Variable(np.array(0.5))
f = Square()
y = f(x)

In [ ]:
def square(x):
  f = Square()
  return f(x)

def exp(x):
  f = Exp()
  return f(x)

In [ ]:
#한줄로표현
def square(x):
  return Square()(x)

def exp(x):
  return Exp()(x)

In [ ]:
#방금 구현한 두 함수 사용
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

In [ ]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))  #연속하여 적용
y.grad = np.array(1.0)
y.backward()
print(x.grad)

In [ ]:
#9.2 backward 메서드 간소화
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator)

In [ ]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

In [ ]:
#9.3 ndarray만 취급하기
class Variable:
  def __init__(self,data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

      self.data = data
      self.grad = None
      self.creator = None

In [ ]:
x = Variable(np.array(1.0)) # OK
x = Variable(None)  # OK

x = Variable(1.0) # NG: 오류 발생 !

In [ ]:
x = np.array([1.0])
y = x ** 2
prit(type(x), x.ndim)
print(type(y))

In [ ]:
x = np.array(1.0) # 0차원 ndarray
y = x ** 2
print(type(x), x.ndim)
print(type(y))

In [ ]:
def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x

In [ ]:
import numpy as np
np.isscalar(np.float64(1.0))
#True
np.isscalar(2.0)
#True
np.isscalar(np.array(1.0))
#False
np.isscalar(np.array([1, 2, 3]))
#False

In [ ]:
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(as_array(y))
    output.set_creator(self)
    self.input = input
    self.output = output
    return output

# Step 10 테스트

In [ ]:
#10.1 파이썬 단위 테스트
import unittest

class SquareTest(unittest.TestCase):
  def test_forward(self):
    x = Variable(np.array(2.0))
    y = square(x)
    expected = np.array(4.0)
    self.assertEqual(y.data, expected)

In [ ]:
#10.2 square 함수의 역전파 테스트
#구현한 테스트 코드에 다음 코드를 추가한다.
import unittest

class SquareTest(unittest.TestCase):
  def test_forward(self):
    x = Variable(np.array(2.0))
    y = square(x)
    expected = np.array(4.0)
    self.assertEqual(y.data, expected)

  def test_backward(self):
    x = Variable(np.array(3.0))
    y = square(x)
    y.backward()
    expected = np.array(6.0)
    self.assertEqual(x.grad, expected)

In [ ]:
#10.3 기울기 확인을 이용한 자동 테스트
import unittest

def numerical_diff(f, x, eps=1e-4):
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / (2 * eps)
  
class SquareTest(unittest.TestCase):

  def test_forward(self):
    x = Variable(np.array(2.0))
    y = square(x)
    expected = np.array(4.0)
    self.assertEqual(y.data, expected)

  def test_backward(self):
    x = Variable(np.array(3.0))
    y = square(x)
    y.backward()
    expected = np.array(6.0)
    self.assertEqual(x.grad, expected)

  def test_gradient_check(self):
    x = Variable(np.random.rand(1))
    y = square(x)
    y.backward()
    num_grad = numerical_diff(square, x)
    flg = np.allclose(x.grad, num_grad)
    self.assertTrue(flg)